In [1]:
import bz2
import json
import pandas as pd
import csv

In [2]:
speaker_attributes = pd.read_parquet('speaker_attributes.parquet')

In [3]:
speaker_attributes.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


In [4]:
label_description = pd.read_csv('wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')

In [5]:
label_description.head()

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America


First let's create an array of all the different IDs for occupation and profession and removing duplicates and None values

In [6]:
work = []
for w in speaker_attributes['occupation'].dropna():
    for i in w:
        work.append(i)
work = list(set(work))

In [7]:
len(work)

14575

Now we will create a dictionnary which will have the QID as keys and the name of the job as values.

In [8]:
dwork = {}


In [9]:
for i in work:
    if i in label_description.index:
        dwork[i]=label_description.loc[i]['Label']

In [10]:
len(dwork)

14558

As some QID had nan values we decided to count them to see how many there were and then we removed them as we couldn't use them.

In [11]:
summ=0
for i in dwork.values():
    if isinstance(i, float):
        summ+=1
        
print(summ)

2460


In [12]:
dic_work={}
for i in dwork:
    if not isinstance(dwork[i], float):
        dic_work[i]=dwork[i]

len(dic_work)

12098

Now we have a new dataset that will help us regroup speaker regarding to their jobs

In [13]:
df_work = pd.DataFrame(dic_work.items(), columns=['ID', 'Label'])

In [18]:
df_work.to_parquet('Id_Work.parquet')

In [19]:
df = pd.read_parquet('Id_Work.parquet')
len(df)

12098

In [20]:
df

,ID,Label
0,Q4305259,air gunner
1,Q98162569,Balsamträger
2,Q41298,magazine
3,Q15132701,Esquire of the Body
4,Q3396098,Fire services in France
...,...,...
12093,Q466118,hetaera
12094,Q1034047,transport planning
12095,Q1208613,diamond cutter
12096,Q107425,landscape
